In [ ]:
#| hide
from longreadtools.core import *

# longreadtools

> A working repository for development of tools for the analasis of long read rna sequencing data.

## Install

```sh
pip install git+https://github.com/cobioda/longreadtools.git

```

In [ ]:
#| hide
!pip install git+https://github.com/cobioda/longreadtools.git

## How to use

Here we will use the `isomatrix_tools` module to convert our isomatrix txt files in bulk into `anndata` objects using the `multiple_isomatrix_conversion` function . Then, we will utilize our specialized `concatenate_anndata` function to generate a concatenated `anndata` for downstream analysis. 


In [ ]:
from longreadtools.isomatrix_tools import *

In [ ]:

# Importing required libraries
import os
import re

# Defining the directory path
directory = '/data/analysis/data_mcandrew/000-sclr-discovair/'

# Defining the regular expression pattern to match the required files
pattern = re.compile('.*(_BIOP_INT|BIOP_NAS)$')

# Getting a list of all files in the directory
all_files = os.listdir(directory)

# Filtering the list to include only files that match the pattern
matching_files = [os.path.join(directory, f) for f in all_files if pattern.match(f)]

# Printing the list of matching files
print(matching_files)

# Assigning the list of matching files to the variable 'individual_runs'
individual_runs = matching_files

# Adding '_isomatrix.txt' to each file name in the 'individual_runs' list
individual_runs = [f'{run}_isomatrix.txt' for run in individual_runs]

# Creating a list of paths for each isomatrix file in the 'matching_files' list
isomatrix_paths = [os.path.join(run, f'{os.path.basename(run)}_isomatrix.txt') for run in matching_files]


In [ ]:
converted_isomatrix_paths = multiple_isomatrix_conversion(isomatrix_paths, verbose=True, return_paths = True, sparse = True)


In [ ]:
converted_isomatrix_paths

In [ ]:
andata_concat = concatenate_anndata(converted_isomatrix_paths)

In [ ]:
andata_concat

In [ ]:
andata_concat.var